# Need to do:

2. Plots/Visualizations
3. Proof read/change comments throughout document
4. run notebook through to only show visualizations
5. Readme (use section titles as guides)
6. Presentation Slides
7. png images? (ask tutor)
8. edit MCForecastTools to change plots as well as print statement. Add plot to show $$ distribution?
9. proofread variables and make sure they are unique to each portfolio
for pie chart: create dataframe with all separate portfolio weights(change labels for all weights to specify which portfolio); also add drop down so you can flip through portfolio weights

# Part I: Imports

In [1]:
# Imports
import os
import requests
import pandas as pd
import numpy as np
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from MCForecastTools import MCSimulation
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime
import seaborn as sns

%matplotlib inline

# Part II: API Data Extraction & Cleansing

In [2]:
# Setting path variable
path = ('/Users/jordanromnes/Downloads/Project_1/keys.env')

In [3]:
# Load .env environment variables
load_dotenv(path)

True

In [4]:
# Setting Alpaca API key and secret
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

In [5]:
# Verifying that Alpaca key and secret were correctly loaded
print(f"Alpaca Key type: {type(alpaca_api_key)}")
print(f"Alpaca Secret Key type: {type(alpaca_secret_key)}")

Alpaca Key type: <class 'str'>
Alpaca Secret Key type: <class 'str'>


In [6]:
# Creating the Alpaca API object
alpaca = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version="v2")

### **** Switch timestamp/corresponding comments ****

In [7]:
# Setting date range for API data extraction
start_date = pd.Timestamp('2021-01-01', tz='America/New_York').isoformat()
end_date = pd.Timestamp('2024-01-01', tz='America/New_York').isoformat()

In [8]:
# Extracting 3 years worth of historical data for each stock
timeframe = "1Day"
ticker = ['AAPL', 'JPM', 'META', 'UNP', 'XOM', 'NEE', 'UNH', 'RIO', 'EQIX', 'WMT', 'AMZN']
stocks_df = alpaca.get_bars(
    ticker,
    timeframe,
    start=start_date,
    end=end_date,
    limit=10000
).df

# Cleaning data so only closing stock prices are retrieved
AAPL = stocks_df[stocks_df['symbol']=='AAPL'].drop(['symbol', 'high', 'low', 'trade_count', 'open', 'volume', 'vwap'], axis=1)
JPM = stocks_df[stocks_df['symbol']=='JPM'].drop(['symbol', 'high', 'low', 'trade_count', 'open', 'volume', 'vwap'], axis=1)
META = stocks_df[stocks_df['symbol']=='META'].drop(['symbol', 'high', 'low', 'trade_count', 'open', 'volume', 'vwap'], axis=1)
UNP = stocks_df[stocks_df['symbol']=='UNP'].drop(['symbol', 'high', 'low', 'trade_count', 'open', 'volume', 'vwap'], axis=1)
XOM = stocks_df[stocks_df['symbol']=='XOM'].drop(['symbol', 'high', 'low', 'trade_count', 'open', 'volume', 'vwap'], axis=1)
NEE = stocks_df[stocks_df['symbol']=='NEE'].drop(['symbol', 'high', 'low', 'trade_count', 'open', 'volume', 'vwap'], axis=1)
UNH = stocks_df[stocks_df['symbol']=='UNH'].drop(['symbol', 'high', 'low', 'trade_count', 'open', 'volume', 'vwap'], axis=1)
RIO = stocks_df[stocks_df['symbol']=='RIO'].drop(['symbol', 'high', 'low', 'trade_count', 'open', 'volume', 'vwap'], axis=1)
EQIX = stocks_df[stocks_df['symbol']=='EQIX'].drop(['symbol', 'high', 'low', 'trade_count', 'open', 'volume', 'vwap'], axis=1)
WMT = stocks_df[stocks_df['symbol']=='WMT'].drop(['symbol', 'high', 'low', 'trade_count', 'open', 'volume', 'vwap'], axis=1)
AMZN = stocks_df[stocks_df['symbol']=='AMZN'].drop(['symbol', 'high', 'low', 'trade_count', 'open', 'volume', 'vwap'], axis=1)

# Concatenating the stock DataFrames
stocks_df = pd.concat([AAPL, JPM, META, UNP, XOM, NEE, UNH, RIO, EQIX, WMT, AMZN],axis=1)

# Setting column names as stock ticker
stocks_df.columns = ['AAPL', 'JPM', 'META', 'UNP', 'XOM', 'NEE', 'UNH', 'RIO', 'EQIX', 'WMT', 'AMZN']
stocks_df.rename_axis('Date', inplace=True)
# Display sample data
stocks_df.head()

,AAPL,JPM,META,UNP,XOM,NEE,UNH,RIO,EQIX,WMT,AMZN
Date,,,,,,,,,,,
2021-01-04 05:00:00+00:00,129.41,125.87,268.94,203.01,41.50,74.22,349.50,77.39,692.56,146.53,3186.63
2021-01-05 05:00:00+00:00,131.01,125.65,270.97,205.50,43.50,74.77,344.80,79.26,691.54,145.75,3218.51
2021-01-06 05:00:00+00:00,126.66,131.55,263.34,209.80,44.61,78.43,359.27,82.80,668.96,146.66,3138.38
2021-01-07 05:00:00+00:00,130.92,135.87,268.74,212.72,44.96,79.29,365.03,85.98,678.97,146.65,3162.16
2021-01-08 05:00:00+00:00,132.05,136.02,267.57,218.82,45.46,81.13,363.39,86.69,692.46,146.63,3182.70


# Part III: Historical Analysis
Contains:

• Daily returns of all stocks composing portfolio. Each stock is representing a sector for the purpose of our experiment/analysis.

• Daily and cumulative returns of each weighted portfolio. Our control portfolio has all 11 stocks weighted evenly(0.09 each). We have 11 other portfolios with 10 stocks weighted evenly (0.075), while one stock is weighted (0.25). Each of the 11 non control portfolios has a different stock weighted more heavily to show the performance of each sector. 

• Historical values of each weighted portfolio. With a given investment of $20,000 at the start date of our historical data. We calculated the historical value of that initial investment today to compare how each "sector/portfolio" performed.

## All Stocks Daily Returns

In [9]:
# Calculating all stocks daily returns
stock_daily_returns = stocks_df.pct_change()
# Dropping NAs
stock_daily_returns = stocks_df.pct_change().dropna()
# Showing sample data
stock_daily_returns.head()

,AAPL,JPM,META,UNP,XOM,NEE,UNH,RIO,EQIX,WMT,AMZN
Date,,,,,,,,,,,
2021-01-05 05:00:00+00:00,0.012364,-0.001748,0.007548,0.012265,0.048193,0.007410,-0.013448,0.024163,-0.001473,-0.005323,0.010004
2021-01-06 05:00:00+00:00,-0.033204,0.046956,-0.028158,0.020925,0.025517,0.048950,0.041966,0.044663,-0.032652,0.006244,-0.024897
2021-01-07 05:00:00+00:00,0.033633,0.032839,0.020506,0.013918,0.007846,0.010965,0.016033,0.038406,0.014964,-0.000068,0.007577
2021-01-08 05:00:00+00:00,0.008631,0.001104,-0.004354,0.028676,0.011121,0.023206,-0.004493,0.008258,0.019868,-0.000136,0.006496
2021-01-11 05:00:00+00:00,-0.023249,0.014924,-0.040102,-0.006718,0.030356,-0.017749,-0.007265,-0.033568,-0.014542,0.004501,-0.021519


In [10]:
# Setting Control Portfolio weights
control_weights = [0.09, 0.09, 0.09, 0.09, 0.09, 0.09, 0.09, 0.09, 0.09, 0.09, 0.09]

# Calculating Control Portfolio daily returns
control_daily_returns = stock_daily_returns.dot(control_weights)
# Showing sample data
control_daily_returns.head()

Date
2021-01-05 05:00:00+00:00    0.008996
2021-01-06 05:00:00+00:00    0.010468
2021-01-07 05:00:00+00:00    0.017696
2021-01-08 05:00:00+00:00    0.008854
2021-01-11 05:00:00+00:00   -0.010344
dtype: float64

## Daily returns, Cumulative returns, historical value of evenly weighted portfolio ("Control")

In [11]:
# Initial investment
initial_investment = 20000

# Calculating control portfolio cumulative returns
control_cumulative_returns = (1 + pd.Series(control_daily_returns)).cumprod()

# Calculating historical value of control portfolio with $20,000 invested at the start of dataframe to the value at the end of dataframe
control_historical_value = initial_investment * control_cumulative_returns


control_historical_value.tail()

Date
2023-12-22 05:00:00+00:00    24720.635167
2023-12-26 05:00:00+00:00    24792.511978
2023-12-27 05:00:00+00:00    24890.990275
2023-12-28 05:00:00+00:00    24915.120241
2023-12-29 05:00:00+00:00    24814.826872
dtype: float64

## Daily returns, Cumulative returns, historical value of tech heavy portfolio ("Technology")

In [12]:
# Setting weights for Technology Portfolio
tech_weights = [0.25, 0.075, 0.075, 0.075, 0.075, 0.075, 0.075, 0.075, 0.075, 0.075, 0.075]
# Calculate Technology portfolio return
tech_daily_returns = stock_daily_returns.dot(tech_weights)
# showing sample data
tech_daily_returns.head()

Date
2021-01-05 05:00:00+00:00    0.009660
2021-01-06 05:00:00+00:00    0.002913
2021-01-07 05:00:00+00:00    0.020632
2021-01-08 05:00:00+00:00    0.008889
2021-01-11 05:00:00+00:00   -0.012688
dtype: float64

In [13]:
# Initial investment
initial_investment = 20000

# Calculating technology portfolio cumulative returns
tech_cumulative_returns = (1 + pd.Series(tech_daily_returns)).cumprod()

# Calculating historical value of technology portfolio with $20,000 invested at the start of dataframe to the value at the end of dataframe
tech_historical_value = initial_investment * tech_cumulative_returns


tech_historical_value.tail()

Date
2023-12-22 05:00:00+00:00    25803.575097
2023-12-26 05:00:00+00:00    25853.267888
2023-12-27 05:00:00+00:00    25941.187925
2023-12-28 05:00:00+00:00    25972.251158
2023-12-29 05:00:00+00:00    25860.473824
dtype: float64

## Daily returns, Cumulative returns, historical value of financial heavy portfolio ("Financial")

In [14]:
# Set weights for financial portfolio
fin_weights = [0.075, 0.25, 0.075, 0.075, 0.075, 0.075, 0.075, 0.075, 0.075, 0.075, 0.075]
# Calculate financial portfolio return
fin_daily_returns = stock_daily_returns.dot(fin_weights)
# showing sample data
fin_daily_returns.head()

Date
2021-01-05 05:00:00+00:00    0.007191
2021-01-06 05:00:00+00:00    0.016941
2021-01-07 05:00:00+00:00    0.020493
2021-01-08 05:00:00+00:00    0.007571
2021-01-11 05:00:00+00:00   -0.006008
dtype: float64

In [15]:
# Initial investment
initial_investment = 20000

# Calculating financial cumulative returns
fin_cumulative_returns = (1 + pd.Series(fin_daily_returns)).cumprod()

# Calculating historical value of financial portfolio with $20,000 invested at the start of dataframe to the value at the end of dataframe
fin_historical_value = initial_investment * fin_cumulative_returns


fin_historical_value.tail()

Date
2023-12-22 05:00:00+00:00    25283.809939
2023-12-26 05:00:00+00:00    25371.239220
2023-12-27 05:00:00+00:00    25481.850939
2023-12-28 05:00:00+00:00    25526.128377
2023-12-29 05:00:00+00:00    25435.254820
dtype: float64

## Daily returns, Cumulative returns, historical value of consumer services heavy portfolio ("Consumer Services")

In [16]:
# Set weights for consumer services portfolio
con_ser_weights = [0.075, 0.075, 0.25, 0.075, 0.075, 0.075, 0.075, 0.075, 0.075, 0.075, 0.075]
# Calculate portfolio return
con_ser_daily_returns = stock_daily_returns.dot(con_ser_weights)
# Display sample data
con_ser_daily_returns.head()

Date
2021-01-05 05:00:00+00:00    0.008818
2021-01-06 05:00:00+00:00    0.003796
2021-01-07 05:00:00+00:00    0.018335
2021-01-08 05:00:00+00:00    0.006616
2021-01-11 05:00:00+00:00   -0.015638
dtype: float64

In [17]:
# Initial investment
initial_investment = 20000

# Calculating cumulative returns
con_ser_cumulative_returns = (1 + pd.Series(con_ser_daily_returns)).cumprod()

# Calculating historical value of consumer services portfolio with $20,000 invested at the start of dataframe to the value at the end of dataframecon_ser_historical_value = initial_investment * con_ser_cumulative_returns


con_ser_historical_value.tail()

NameError: name 'con_ser_historical_value' is not defined

## Daily returns, Cumulative returns, historical value of industrial heavy portfolio ("Industrial")

In [ ]:
# Set weights
ind_weights = [0.075, 0.075, 0.075, 0.25, 0.075, 0.075, 0.075, 0.075, 0.075, 0.075, 0.075]
# Calculate portfolio return
ind_daily_returns = stock_daily_returns.dot(ind_weights)
# Display sample data
ind_daily_returns.head()

In [ ]:
# Initial investment
initial_investment = 20000

# Calculating cumulative returns
ind_cumulative_returns = (1 + pd.Series(ind_daily_returns)).cumprod()

# Calculating historical value of industrial portfolio with $20,000 invested at the start of dataframe to the value at the end of dataframeind_historical_value = initial_investment * ind_cumulative_returns


ind_historical_value.tail()

## Daily returns, Cumulative returns, historical value of energy heavy portfolio ("Energy")

In [ ]:
# Set weights
nrg_weights = [0.075, 0.075, 0.075, 0.075, 0.25, 0.075, 0.075, 0.075, 0.075, 0.075, 0.075]
# Calculate portfolio return
nrg_daily_returns = stock_daily_returns.dot(nrg_weights)
# Display sample data
nrg_daily_returns.head()

In [ ]:
# Initial investment
initial_investment = 20000

# Calculating cumulative returns
nrg_cumulative_returns = (1 + pd.Series(nrg_daily_returns)).cumprod()

# Calculating historical value of energy portfolio with $20,000 invested at the start of dataframe to the value at the end of dataframenrg_historical_value = initial_investment * nrg_cumulative_returns


nrg_historical_value.tail()

## Daily returns, Cumulative returns, historical value of utilities heavy portfolio ("Utilities")

In [ ]:
# Set weights
util_weights = [0.075, 0.075, 0.075, 0.075, 0.075, 0.25, 0.075, 0.075, 0.075, 0.075, 0.075]
# Calculate portfolio return
util_daily_returns = stock_daily_returns.dot(util_weights)
# Display sample data
util_daily_returns.head()

In [ ]:
# Initial investment
initial_investment = 20000

# Calculating cumulative returns
util_cumulative_returns = (1 + pd.Series(util_daily_returns)).cumprod()

# Calculating historical value of utilities portfolio with $20,000 invested at the start of dataframe to the value at the end of dataframeutil_historical_value = initial_investment * util_cumulative_returns


util_historical_value.tail()

## Daily returns, Cumulative returns, historical value of healthcare heavy portfolio ("Healthcare")

In [ ]:
# Set weights
hc_weights = [0.075, 0.075, 0.075, 0.075, 0.075, 0.075, 0.25, 0.075, 0.075, 0.075, 0.075]
# Calculate portfolio return
hc_daily_returns = stock_daily_returns.dot(hc_weights)
# Display sample data
hc_daily_returns.head()

In [ ]:
# Initial investment
initial_investment = 20000

# Calculating cumulative returns
hc_cumulative_returns = (1 + pd.Series(hc_daily_returns)).cumprod()

# Calculating historical value of healthcare portfolio with $20,000 invested at the start of dataframe to the value at the end of dataframehc_historical_value = initial_investment * hc_cumulative_returns


hc_historical_value.tail()

## Daily returns, Cumulative returns, historical value of materials heavy portfolio ("Materials")

In [ ]:
# Set weights
mat_weights = [0.075, 0.075, 0.075, 0.075, 0.075, 0.075, 0.075, 0.25, 0.075, 0.075, 0.075]
# Calculate portfolio return
mat_daily_returns = stock_daily_returns.dot(mat_weights)
# Display sample data
mat_daily_returns.head()

In [ ]:
# Initial investment
initial_investment = 20000

# Calculating cumulative returns
mat_cumulative_returns = (1 + pd.Series(mat_daily_returns)).cumprod()

# Calculating historical value of materials portfolio with $20,000 invested at the start of dataframe to the value at the end of dataframemat_historical_value = initial_investment * mat_cumulative_returns


mat_historical_value.tail()

## Daily returns, Cumulative returns, historical value of real estate heavy portfolio ("Real Estate")

In [ ]:
# Set weights
re_weights = [0.075, 0.075, 0.075, 0.075, 0.075, 0.075, 0.075, 0.075, 0.25, 0.075, 0.075]
# Calculate portfolio return
re_daily_returns = stock_daily_returns.dot(re_weights)
# Display sample data
re_daily_returns.head()

In [ ]:
# Initial investment
initial_investment = 20000

# Calculating cumulative returns
re_cumulative_returns = (1 + pd.Series(re_daily_returns)).cumprod()

# Calculating historical value of real estate portfolio with $20,000 invested at the start of dataframe to the value at the end of dataframere_historical_value = initial_investment * re_cumulative_returns


re_historical_value.tail()

## Daily returns, Cumulative returns, historical value of consumer staples heavy portfolio ("Consumer Staples")

In [ ]:
# Set weights
con_staple_weights = [0.075, 0.075, 0.075, 0.075, 0.075, 0.075, 0.075, 0.075, 0.075, 0.25, 0.075]
# Calculate portfolio return
con_staple_daily_returns = stock_daily_returns.dot(con_staple_weights)
# Display sample data
con_staple_daily_returns.head()

In [ ]:
# Initial investment
initial_investment = 20000

# Calculating cumulative returns
con_staple_cumulative_returns = (1 + pd.Series(con_staple_daily_returns)).cumprod()

# Calculating historical value of consumer staples portfolio with $20,000 invested at the start of dataframe to the value at the end of dataframecon_staple_historical_value = initial_investment * con_staple_cumulative_returns


con_staple_historical_value.tail()

## Daily returns, Cumulative returns, historical value of consumer discretionary heavy portfolio ("Consumer Discretionary")

In [ ]:
# Set weights
con_disc_weights = [0.075, 0.075, 0.075, 0.075, 0.075, 0.075, 0.075, 0.075, 0.075, 0.075, 0.25]
# Calculate portfolio return
con_disc_daily_returns = stock_daily_returns.dot(con_disc_weights)
# Display sample data
con_disc_daily_returns.head()

In [ ]:
# Initial investment
initial_investment = 20000

# Calculating cumulative returns
con_disc_cumulative_returns = (1 + pd.Series(con_disc_daily_returns)).cumprod()

# Calculating historical value of consumer discretionary portfolio with $20,000 invested at the start of dataframe to the value at the end of dataframecon_disc_historical_value = initial_investment * con_disc_cumulative_returns


con_disc_historical_value.tail()

## Concatenating portfolio returns for plotting

### Combining/concatenating all historical values(cumulative returns * initial investment) of all portfolios with an initial investment of $20,000 from 1/05/2021 through 12/29/2023
*Need Line plot*

In [ ]:
all_historical_values_df = pd.concat([
    control_historical_value, 
    tech_historical_value,
    fin_historical_value,
    con_ser_historical_value,
    ind_historical_value,
    nrg_historical_value,
    util_historical_value,
    hc_historical_value,
    mat_historical_value,
    re_historical_value,
    con_staple_historical_value,
    con_disc_historical_value], keys=['Control', 'Technology', 'Finance', 'Consumer Services', 'Industrial', 'Energy', 'Utilities', 'Healthcare', 'Materials', 'Real Estate', 'Consumer Staples', 'Consumer Discretionary'], axis=1)

all_historical_values_df

### Combining/concatenating all cumulative returns of all portfolios from 1/05/2021 through 12/29/2023

*Need Line plot*

In [ ]:
all_cumulative_returns_df = pd.concat([
    control_cumulative_returns, 
    tech_cumulative_returns,
    fin_cumulative_returns,
    con_ser_cumulative_returns,
    ind_cumulative_returns,
    nrg_cumulative_returns,
    util_cumulative_returns,
    hc_cumulative_returns,
    mat_cumulative_returns,
    re_cumulative_returns,
    con_staple_cumulative_returns,
    con_disc_cumulative_returns], keys=['Control', 'Technology', 'Finance', 'Consumer Services', 'Industrial', 'Energy', 'Utilities', 'Healthcare', 'Materials', 'Real Estate', 'Consumer Staples', 'Consumer Discretionary'], axis=1)

all_cumulative_returns_df

### Combining/concatenating all daily returns of all portfolios from 1/05/2021 through 12/29/2023

*Need Line plot*

In [ ]:
all_daily_returns_df = pd.concat([
    control_daily_returns, 
    tech_daily_returns,
    fin_daily_returns,
    con_ser_daily_returns,
    ind_daily_returns,
    nrg_daily_returns,
    util_daily_returns,
    hc_daily_returns,
    mat_daily_returns,
    re_daily_returns,
    con_staple_daily_returns,
    con_disc_daily_returns], keys=['Control', 'Technology', 'Finance', 'Consumer Services', 'Industrial', 'Energy', 'Utilities', 'Healthcare', 'Materials', 'Real Estate', 'Consumer Staples', 'Consumer Discretionary'], axis=1)

all_daily_returns_df

## Plot cumulative & daily returns alongside historical values (Thursday/Monday)

# Part IV: Risk Analysis

In [ ]:
## Use all_returns_df for Risk Analysis
all_portfolio_std = all_daily_returns_df.std()
all_portfolio_std.sort_values()

In [ ]:
## Determine which portfolios are riskier
volatility = all_daily_returns_df.std() * np.sqrt(252)
volatility.sort_values()

*Add more detail to box plot*

In [ ]:
## Create a box plot for each of the returns.

# Box plot to visually show risk

all_daily_returns_df.boxplot(column=['Control', 'Technology', 'Finance', 'Consumer Services', 'Industrial', 'Energy', 'Utilities', 'Healthcare', 'Materials', 'Real Estate', 'Consumer Staples', 'Consumer Discretionary'], figsize=(20,20), fontsize='large', rot=45)

In [ ]:
## Calculate the Annualized Standard Deviation.
annualized_std = all_daily_returns_df.std() * np.sqrt(252)
annualized_std.sort_values()

# Part V: Monte Carlo 5 year simulation with an initial investment of $20,000 for each portfolio

## Control 5 Year MC

In [ ]:
# Configuring a Monte Carlo simulation to forecast 5 years cumulative returns
MC_5_year_control = MCSimulation(
    portfolio_data = stocks_df,
    weights = [0.09, 0.09, 0.09, 0.09, 0.09, 0.09, 0.09, 0.09, 0.09, 0.09, 0.09],
    num_simulation = 500,
    num_trading_days = 252*5
)

In [ ]:
# Running a Monte Carlo simulation to forecast 5 years cumulative returns
MC_5_year_control.calc_cumulative_return()

In [ ]:
# Plot simulation outcomes
five_year_line_plot_control = MC_5_year_control.plot_simulation()

In [ ]:
# Plot probability distribution and confidence intervals
five_year_dist_plot_control = MC_5_year_control.plot_distribution()

In [ ]:
# Fetch summary statistics from the Monte Carlo simulation results
five_year_summary_control = MC_5_year_control.summarize_cumulative_return()

# Print summary statistics
print(five_year_summary_control)

In [ ]:
# Set initial investment
initial_investment = 20000

# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes of our $20,000
ci_lower_five_control = round(five_year_summary_control.iloc[8]*initial_investment,2)
ci_upper_five_control = round(five_year_summary_control.iloc[9]*initial_investment,2)

# Print results
print(f"There is a 95% chance that an initial investment of ${initial_investment} in the portfolio"
      f" over the next 5 years will end within in the range of"
      f" ${ci_lower_five_control} and ${ci_upper_five_control}")

## Tech

In [ ]:
MC_5_year_tech = MCSimulation(
    portfolio_data = stocks_df,
    weights = [0.25, 0.075, 0.075, 0.075, 0.075, 0.075, 0.075, 0.075, 0.075, 0.075, 0.075],
    num_simulation = 500,
    num_trading_days = 252*5
)

In [ ]:
# Running a Monte Carlo simulation to forecast 5 years cumulative returns
MC_5_year_tech.calc_cumulative_return()

In [ ]:
# Plot simulation outcomes
five_year_line_plot_tech = MC_5_year_tech.plot_simulation()

In [ ]:
# Plot probability distribution and confidence intervals
five_year_dist_plot_tech = MC_5_year_tech.plot_distribution()

In [ ]:
# Fetch summary statistics from the Monte Carlo simulation results
five_year_summary_tech = MC_5_year_tech.summarize_cumulative_return()

# Print summary statistics
print(five_year_summary_tech)

In [ ]:
# Set initial investment
initial_investment = 20000

# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes of our $20,000
ci_lower_five_tech = round(five_year_summary_tech.iloc[8]*initial_investment,2)
ci_upper_five_tech = round(five_year_summary_tech.iloc[9]*initial_investment,2)

# Print results
print(f"There is a 95% chance that an initial investment of ${initial_investment} in the portfolio"
      f" over the next 5 years will end within in the range of"
      f" ${ci_lower_five_tech} and ${ci_upper_five_tech}")

## Fin

In [ ]:
MC_5_year_fin = MCSimulation(
    portfolio_data = stocks_df,
    weights = [0.075, 0.25, 0.075, 0.075, 0.075, 0.075, 0.075, 0.075, 0.075, 0.075, 0.075],
    num_simulation = 500,
    num_trading_days = 252*5
)

In [ ]:
# Running a Monte Carlo simulation to forecast 5 years cumulative returns
MC_5_year_fin.calc_cumulative_return()

In [ ]:
# Plot simulation outcomes
five_year_line_plot_fin = MC_5_year_fin.plot_simulation()

In [ ]:
# Plot probability distribution and confidence intervals
five_year_dist_plot_fin = MC_5_year_fin.plot_distribution()

In [ ]:
# Fetch summary statistics from the Monte Carlo simulation results
five_year_summary_fin = MC_5_year_fin.summarize_cumulative_return()

# Print summary statistics
print(five_year_summary_fin)

In [ ]:
# Set initial investment
initial_investment = 20000

# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes of our $20,000
ci_lower_five_fin = round(five_year_summary_fin.iloc[8]*initial_investment,2)
ci_upper_five_fin = round(five_year_summary_fin.iloc[9]*initial_investment,2)

# Print results
print(f"There is a 95% chance that an initial investment of ${initial_investment} in the portfolio"
      f" over the next 5 years will end within in the range of"
      f" ${ci_lower_five_fin} and ${ci_upper_five_fin}")

## Con_ser

In [ ]:
MC_5_year_con_ser = MCSimulation(
    portfolio_data = stocks_df,
    weights = [0.075, 0.075, 0.25, 0.075, 0.075, 0.075, 0.075, 0.075, 0.075, 0.075, 0.075],
    num_simulation = 500,
    num_trading_days = 252*5
)

In [ ]:
# Running a Monte Carlo simulation to forecast 5 years cumulative returns
MC_5_year_con_ser.calc_cumulative_return()

In [ ]:
# Plot simulation outcomes
five_year_line_plot_con_ser = MC_5_year_con_ser.plot_simulation()

In [ ]:
# Plot probability distribution and confidence intervals
five_year_dist_plot_con_ser = MC_5_year_con_ser.plot_distribution()

In [ ]:
# Fetch summary statistics from the Monte Carlo simulation results
five_year_summary_con_ser = MC_5_year_con_ser.summarize_cumulative_return()

# Print summary statistics
print(five_year_summary_con_ser)

In [ ]:
# Set initial investment
initial_investment = 20000

# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes of our $20,000
ci_lower_five_con_ser = round(five_year_summary_con_ser.iloc[8]*initial_investment,2)
ci_upper_five_con_ser = round(five_year_summary_con_ser.iloc[9]*initial_investment,2)

# Print results
print(f"There is a 95% chance that an initial investment of ${initial_investment} in the portfolio"
      f" over the next 5 years will end within in the range of"
      f" ${ci_lower_five_con_ser} and ${ci_upper_five_con_ser}")

## Industrial

In [ ]:
MC_5_year_ind= MCSimulation(
    portfolio_data = stocks_df,
    weights = [0.075, 0.075, 0.075, 0.25, 0.075, 0.075, 0.075, 0.075, 0.075, 0.075, 0.075],
    num_simulation = 500,
    num_trading_days = 252*5
)

In [ ]:
# Running a Monte Carlo simulation to forecast 5 years cumulative returns
MC_5_year_ind.calc_cumulative_return()

In [ ]:
# Plot simulation outcomes
five_year_line_plot_ind = MC_5_year_ind.plot_simulation()

In [ ]:
# Plot probability distribution and confidence intervals
five_year_dist_plot_ind = MC_5_year_ind.plot_distribution()

In [ ]:
# Fetch summary statistics from the Monte Carlo simulation results
five_year_summary_ind = MC_5_year_ind.summarize_cumulative_return()

# Print summary statistics
print(five_year_summary_ind)

In [ ]:
# Set initial investment
initial_investment = 20000

# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes of our $20,000
ci_lower_five_ind = round(five_year_summary_ind.iloc[8]*initial_investment,2)
ci_upper_five_ind = round(five_year_summary_ind.iloc[9]*initial_investment,2)

# Print results
print(f"There is a 95% chance that an initial investment of ${initial_investment} in the portfolio"
      f" over the next 5 years will end within in the range of"
      f" ${ci_lower_five_ind} and ${ci_upper_five_ind}")

## energy

In [ ]:
MC_5_year_nrg = MCSimulation(
    portfolio_data = stocks_df,
    weights = [0.075, 0.075, 0.075, 0.075, 0.25, 0.075, 0.075, 0.075, 0.075, 0.075, 0.075],
    num_simulation = 500,
    num_trading_days = 252*5
)

In [ ]:
# Running a Monte Carlo simulation to forecast 5 years cumulative returns
MC_5_year_nrg.calc_cumulative_return()

In [ ]:
# Plot simulation outcomes
five_year_line_plot_nrg = MC_5_year_nrg.plot_simulation()

In [ ]:
# Plot probability distribution and confidence intervals
five_year_dist_plot_nrg = MC_5_year_nrg.plot_distribution()

In [ ]:
# Fetch summary statistics from the Monte Carlo simulation results
five_year_summary_nrg = MC_5_year_nrg.summarize_cumulative_return()

# Print summary statistics
print(five_year_summary_nrg)

In [ ]:
# Set initial investment
initial_investment = 20000

# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes of our $20,000
ci_lower_five_control = round(five_year_summary_nrg.iloc[8]*initial_investment,2)
ci_upper_five_control = round(five_year_summary_nrg.iloc[9]*initial_investment,2)

# Print results
print(f"There is a 95% chance that an initial investment of ${initial_investment} in the portfolio"
      f" over the next 5 years will end within in the range of"
      f" ${ci_lower_five_control} and ${ci_upper_five_control}")

## utilities

In [ ]:
MC_5_year_util = MCSimulation(
    portfolio_data = stocks_df,
    weights = [0.075, 0.075, 0.075, 0.075, 0.075, 0.25, 0.075, 0.075, 0.075, 0.075, 0.075],
    num_simulation = 500,
    num_trading_days = 252*5
)

In [ ]:
# Running a Monte Carlo simulation to forecast 5 years cumulative returns
MC_5_year_util.calc_cumulative_return()

In [ ]:
# Plot simulation outcomes
five_year_line_plot_util = MC_5_year_util.plot_simulation()

In [ ]:
# Plot probability distribution and confidence intervals
five_year_dist_plot_util = MC_5_year_util.plot_distribution()

In [ ]:
# Fetch summary statistics from the Monte Carlo simulation results
five_year_summary_util = MC_5_year_util.summarize_cumulative_return()

# Print summary statistics
print(five_year_summary_util)

In [ ]:
# Set initial investment
initial_investment = 20000

# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes of our $20,000
ci_lower_five_util = round(five_year_summary_util.iloc[8]*initial_investment,2)
ci_upper_five_util = round(five_year_summary_util.iloc[9]*initial_investment,2)

# Print results
print(f"There is a 95% chance that an initial investment of ${initial_investment} in the portfolio"
      f" over the next 5 years will end within in the range of"
      f" ${ci_lower_five_util} and ${ci_upper_five_util}")

## Healthcare

In [ ]:
MC_5_year_hc = MCSimulation(
    portfolio_data = stocks_df,
    weights = [0.075, 0.075, 0.075, 0.075, 0.075, 0.075, 0.25, 0.075, 0.075, 0.075, 0.075],
    num_simulation = 500,
    num_trading_days = 252*5
)

In [ ]:
# Running a Monte Carlo simulation to forecast 5 years cumulative returns
MC_5_year_hc.calc_cumulative_return()

In [ ]:
# Plot simulation outcomes
five_year_line_plot_hc = MC_5_year_hc.plot_simulation()

In [ ]:
# Plot probability distribution and confidence intervals
five_year_dist_plot_hc = MC_5_year_hc.plot_distribution()

In [ ]:
# Fetch summary statistics from the Monte Carlo simulation results
five_year_summary_hc = MC_5_year_hc.summarize_cumulative_return()

# Print summary statistics
print(five_year_summary_hc)

In [ ]:
# Set initial investment
initial_investment = 20000

# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes of our $20,000
ci_lower_five_hc = round(five_year_summary_hc.iloc[8]*initial_investment,2)
ci_upper_five_hc = round(five_year_summary_hc.iloc[9]*initial_investment,2)

# Print results
print(f"There is a 95% chance that an initial investment of ${initial_investment} in the portfolio"
      f" over the next 5 years will end within in the range of"
      f" ${ci_lower_five_hc} and ${ci_upper_five_hc}")

## Materials

In [ ]:
MC_5_year_mat = MCSimulation(
    portfolio_data = stocks_df,
    weights = [0.075, 0.075, 0.075, 0.075, 0.075, 0.075, 0.075, 0.25, 0.075, 0.075, 0.075],
    num_simulation = 500,
    num_trading_days = 252*5
)

In [ ]:
# Running a Monte Carlo simulation to forecast 5 years cumulative returns
MC_5_year_mat.calc_cumulative_return()

In [ ]:
# Plot simulation outcomes
five_year_line_plot_mat = MC_5_year_mat.plot_simulation()

In [ ]:
# Plot probability distribution and confidence intervals
five_year_dist_plot_mat = MC_5_year_mat.plot_distribution()

In [ ]:
# Fetch summary statistics from the Monte Carlo simulation results
five_year_summary_mat = MC_5_year_mat.summarize_cumulative_return()

# Print summary statistics
print(five_year_summary_mat)

In [ ]:
# Set initial investment
initial_investment = 20000

# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes of our $20,000
ci_lower_five_mat = round(five_year_summary_mat.iloc[8]*initial_investment,2)
ci_upper_five_mat = round(five_year_summary_mat.iloc[9]*initial_investment,2)

# Print results
print(f"There is a 95% chance that an initial investment of ${initial_investment} in the portfolio"
      f" over the next 5 years will end within in the range of"
      f" ${ci_lower_five_mat} and ${ci_upper_five_mat}")

## Real Estate

In [ ]:
MC_5_year_re = MCSimulation(
    portfolio_data = stocks_df,
    weights = [0.075, 0.075, 0.075, 0.075, 0.075, 0.075, 0.075, 0.075, 0.25, 0.075, 0.075],
    num_simulation = 500,
    num_trading_days = 252*5
)


In [ ]:
# Running a Monte Carlo simulation to forecast 5 years cumulative returns
MC_5_year_re.calc_cumulative_return()

In [ ]:
# Plot simulation outcomes
five_year_line_plot_re = MC_5_year_re.plot_simulation()

In [ ]:
# Plot probability distribution and confidence intervals
five_year_dist_plot_re = MC_5_year_re.plot_distribution()

In [ ]:
# Fetch summary statistics from the Monte Carlo simulation results
five_year_summary_re = MC_5_year_re.summarize_cumulative_return()

# Print summary statistics
print(five_year_summary_re)

In [ ]:
# Set initial investment
initial_investment = 20000

# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes of our $20,000
ci_lower_five_re = round(five_year_summary_re.iloc[8]*initial_investment,2)
ci_upper_five_re = round(five_year_summary_re.iloc[9]*initial_investment,2)

# Print results
print(f"There is a 95% chance that an initial investment of ${initial_investment} in the portfolio"
      f" over the next 5 years will end within in the range of"
      f" ${ci_lower_five_re} and ${ci_upper_five_re}")

## Consumer Staples

In [ ]:
MC_5_year_con_staple = MCSimulation(
    portfolio_data = stocks_df,
    weights = [0.075, 0.075, 0.075, 0.075, 0.075, 0.075, 0.075, 0.075, 0.075, 0.25, 0.075],
    num_simulation = 500,
    num_trading_days = 252*5
)

In [ ]:
# Running a Monte Carlo simulation to forecast 5 years cumulative returns
MC_5_year_con_staple.calc_cumulative_return()

In [ ]:
# Plot simulation outcomes
five_year_line_plot_con_staple = MC_5_year_con_staple.plot_simulation()

In [ ]:
# Plot probability distribution and confidence intervals
five_year_dist_plot_con_staple = MC_5_year_con_staple.plot_distribution()

In [ ]:
# Fetch summary statistics from the Monte Carlo simulation results
five_year_summary_con_staple = MC_5_year_con_staple.summarize_cumulative_return()

# Print summary statistics
print(five_year_summary_con_staple)

In [ ]:
# Set initial investment
initial_investment = 20000

# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes of our $20,000
ci_lower_five_con_staple = round(five_year_summary_con_staple.iloc[8]*initial_investment,2)
ci_upper_five_con_staple = round(five_year_summary_con_staple.iloc[9]*initial_investment,2)

# Print results
print(f"There is a 95% chance that an initial investment of ${initial_investment} in the portfolio"
      f" over the next 5 years will end within in the range of"
      f" ${ci_lower_five_con_staple} and ${ci_upper_five_con_staple}")

## Consumer Discretionary

In [ ]:
MC_5_year_con_disc = MCSimulation(
    portfolio_data = stocks_df,
    weights = [0.075, 0.075, 0.075, 0.075, 0.075, 0.075, 0.075, 0.075, 0.25, 0.075, 0.25],
    num_simulation = 500,
    num_trading_days = 252*5
)

In [ ]:
# Running a Monte Carlo simulation to forecast 5 years cumulative returns
MC_5_year_con_disc.calc_cumulative_return()

In [ ]:
# Plot simulation outcomes
five_year_line_plot_con_disc = MC_5_year_con_disc.plot_simulation()

In [ ]:
# Plot probability distribution and confidence intervals
five_year_dist_plot_con_disc = MC_5_year_con_disc.plot_distribution()

In [ ]:
# Fetch summary statistics from the Monte Carlo simulation results
five_year_summary_con_disc = MC_5_year_con_disc.summarize_cumulative_return()

# Print summary statistics
print(five_year_summary_con_disc)

In [ ]:
# Set initial investment
initial_investment = 20000

# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes of our $20,000
ci_lower_five_con_disc = round(five_year_summary_con_disc.iloc[8]*initial_investment,2)
ci_upper_five_con_disc = round(five_year_summary_con_disc.iloc[9]*initial_investment,2)

# Print results
print(f"There is a 95% chance that an initial investment of ${initial_investment} in the portfolio"
      f" over the next 5 years will end within in the range of"
      f" ${ci_lower_five_con_disc} and ${ci_upper_five_con_disc}")

# Part VI: Plots/Visualizations

Plots Needed:
1. Historical analysis(All Line Charts): daily returns, historical values, and cumulative returns
2. Risk Analysis: Boxplot
3. Monte Carlo: box plot of summary stats
4. Pie/Donut Charts represent each portfolios weighted distribution?

*Pie/donut chart showing each portfolio's distribution

## Historical Cumulative Returns

In [ ]:
hist_cumulative_returns_fig = px.line(all_cumulative_returns_df, width=800, height=500, title="Cumulative Returns of all Portfolios")
hist_cumulative_returns_fig.show()

## Portfolios Historical Results with an Initial Investment of $20,000

In [ ]:
historical_values_fig = px.line(all_historical_values_df, width=1000, height=850, title="Historical Value with an initial investment of $20,000 of all Portfolios")
historical_values_fig.show()

## Portfolios Daily Returns

In [ ]:
hist_daily_returns_fig = px.line(all_daily_returns_df, width=1000, height=850, title="Daily Returns of all Portfolios")
hist_daily_returns_fig.show()

## Candlestick/boxplot chart to show Monte Carlo simulation summary statistics values (minimum, maximum, CI upper, CI lower)

In [ ]:
# Plotting the upper and lower CI 95% of all MC simulation(min/max = high/low; open/close = CIupper/CIlower
min_data = [0.612243, 0.805576, 0.786399, 0.604798, 0.769029, 0.862505, 0.649108, 0.909431, 0.677289, 0.647386, 0.628008, 0.280719]
max_data = [2.967900, 3.577134, 3.090839, 4.066498, 3.077694, 3.639074, 2.797470, 3.107733, 3.281936, 3.183344, 2.978899, 4.169834]
ci_upper = [2.363665, 2.707885, 2.420722, 2.917650, 2.374532, 2.917394, 2.165186, 2.553763, 2.301589, 2.331461, 2.292041, 2.457851]
ci_lower = [0.949598, 1.015959, 1.030039, 0.964470, 0.958643, 1.173393, 0.876502, 1.047440, 0.892707, 0.924784, 0.923972, 0.450766]
portfolios = ['Control', 'Technology', 'Finance', 'Consumer Services', 'Industrial', 'Energy', 'Utilities', 'Healthcare', 'Materials', 'Real Estate', 'Consumer Staples', 'Consumer Discretionary']

candle_fig = go.Figure(data=[go.Candlestick(x=portfolios,
                       open=ci_upper, high=max_data,
                       low=min_data, close=ci_lower)])

candle_fig.show()

## Box plot for risk calculated with historical data using std

In [ ]:
hist_risk_fig = px.box(all_daily_returns_df, width=1000, height=850)
hist_risk_fig.show()

## Box plot for risk calculated with Monte Carlo sim data using summary statistics

In [ ]:
#mc_risk_fig = px.box()
#mc_risk_fig.show()

## Pie Chart to display portfolio weights

In [ ]:
#fig = px.pie(weights)
#fig.show()

## Histogram to plot distribution (edit MC file)

In [ ]:
#fig = px.histogram(df, x="total_bill")
#fig.show()

## Heatmap of correlation of stocks within portfolios

In [ ]:
#stocks_correlation = stocks_df.corr()
#stocks_correlation_fig = px.imshow(stocks_correlation)
#stocks_correlation_fig.show()